---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']# Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df.fillna(-1,inplace=True)
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['between'] = pd.Series(nx.betweenness_centrality(G))
    train_df = df[df['ManagementSalary']!= -1]

    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_df[['Department','clustering', 'degree','closeness','between']], train_df['ManagementSalary'])
    test_df=train_df = df[df['ManagementSalary']== -1]
    y_pred = clf.predict_proba(test_df[['Department','clustering', 'degree','closeness','between']])
    
    return pd.Series([x[0] for x in y_pred[:,1:]], index=test_df.index)# Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    
    # Your Code Here
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    future_connections['source'] = [source for source,dest in future_connections.index]
    future_connections['dest'] = [dest for source,dest in future_connections.index]
    fc_indices = future_connections.copy().index
    future_connections.fillna(-1,inplace=True)
    for x in G.nodes(data=True):
        x[1]['community'] = x[1]['Department']


    future_connections['common_neighbors'] = [len(list(nx.common_neighbors(G,n0,n1))) for n0,n1 in future_connections.index]

    jaccard_coefficients = list(nx.jaccard_coefficient(G))
    jacard_df = pd.DataFrame([item2 for item0,item1,item2 in jaccard_coefficients], columns=['jaccard_coefficient'])
    jacard_df['source'] = [item0 for item0,item1,item2 in jaccard_coefficients]
    jacard_df['dest'] = [item1 for item0,item1,item2 in jaccard_coefficients]
    future_connections = future_connections.merge(jacard_df, on=['source','dest'], how='inner')

    resource_allocation_index = list(nx.resource_allocation_index(G))
    resource_alloc_df = pd.DataFrame([item2 for item0,item1,item2 in resource_allocation_index], columns=['resource_allocation_index'])
    resource_alloc_df['source'] = [item0 for item0,item1,item2 in resource_allocation_index]
    resource_alloc_df['dest'] = [item1 for item0,item1,item2 in resource_allocation_index]
    future_connections = future_connections.merge(resource_alloc_df, on=['source','dest'], how='inner')

    adamic_adar_index = list(nx.adamic_adar_index(G))
    adamic_adar_index_df = pd.DataFrame([item2 for item0,item1,item2 in adamic_adar_index], columns=['adamic_adar_index'])
    adamic_adar_index_df['source'] = [item0 for item0,item1,item2 in adamic_adar_index]
    adamic_adar_index_df['dest'] = [item1 for item0,item1,item2 in adamic_adar_index]
    future_connections = future_connections.merge(adamic_adar_index_df, on=['source','dest'], how='inner')

    preferential_attachment = list(nx.preferential_attachment(G))
    preferential_attachment_df = pd.DataFrame([item2 for item0,item1,item2 in preferential_attachment], columns=['preferential_attachment'])
    preferential_attachment_df['source'] = [item0 for item0,item1,item2 in preferential_attachment]
    preferential_attachment_df['dest'] = [item1 for item0,item1,item2 in preferential_attachment]
    future_connections = future_connections.merge(preferential_attachment_df, on=['source','dest'], how='inner')

    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    cn_soundarajan_hopcroft_df = pd.DataFrame([item2 for item0,item1,item2 in cn_soundarajan_hopcroft], columns=['cn_soundarajan_hopcroft'])
    cn_soundarajan_hopcroft_df['source'] = [item0 for item0,item1,item2 in cn_soundarajan_hopcroft]
    cn_soundarajan_hopcroft_df['dest'] = [item1 for item0,item1,item2 in cn_soundarajan_hopcroft]
    future_connections = future_connections.merge(cn_soundarajan_hopcroft_df, on=['source','dest'], how='inner')

    ra_index_soundarajan_hopcroft = list(nx.ra_index_soundarajan_hopcroft(G))
    ra_index_soundarajan_hopcroft_df = pd.DataFrame([item2 for item0,item1,item2 in ra_index_soundarajan_hopcroft], columns=['ra_index_soundarajan_hopcroft'])
    ra_index_soundarajan_hopcroft_df['source'] = [item0 for item0,item1,item2 in ra_index_soundarajan_hopcroft]
    ra_index_soundarajan_hopcroft_df['dest'] = [item1 for item0,item1,item2 in ra_index_soundarajan_hopcroft]
    future_connections = future_connections.merge(ra_index_soundarajan_hopcroft_df, on=['source','dest'], how='inner')


    future_connections['fc_indices'] = fc_indices
    future_connections.reset_index(inplace=True)
    future_connections.set_index ('fc_indices', inplace=True)
    future_connections.drop('index',axis=1 ,inplace=True)

    train_df = future_connections[future_connections['Future Connection'] != -1]
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_df[['common_neighbors','jaccard_coefficient', 'resource_allocation_index', 'adamic_adar_index','preferential_attachment', 'cn_soundarajan_hopcroft','ra_index_soundarajan_hopcroft']], train_df['Future Connection'])
    test_df=  future_connections[future_connections['Future Connection']== -1]
    y_pred = clf.predict_proba(test_df[['common_neighbors','jaccard_coefficient', 'resource_allocation_index', 'adamic_adar_index','preferential_attachment', 'cn_soundarajan_hopcroft','ra_index_soundarajan_hopcroft']])


    return pd.Series([x[0] for x in y_pred[:,1:]], index=test_df.index)# Your Answer Here
new_connections_predictions()

fc_indices
(107, 348)    0.000000
(542, 751)    0.012934
(20, 426)     0.200000
(50, 989)     0.014211
(942, 986)    0.011175
(324, 857)    0.016824
(13, 710)     0.100000
(19, 271)     0.000000
(319, 878)    0.012455
(659, 707)    0.013977
(49, 843)     0.014720
(208, 893)    0.005787
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.100000
(239, 689)    0.006644
(359, 373)    0.164383
(53, 523)     0.400000
(276, 984)    0.007023
(202, 997)    0.010857
(604, 619)    0.000000
(270, 911)    0.011734
(261, 481)    0.000000
(200, 450)    1.000000
(213, 634)    0.013454
(644, 735)    0.000000
(346, 553)    0.000000
(521, 738)    0.019938
(422, 953)    0.000000
                ...   
(672, 848)    0.011734
(28, 127)     1.000000
(202, 661)    0.011998
(54, 195)     1.000000
(295, 864)    0.014343
(814, 936)    0.012551
(839, 874)    0.011175
(139, 843)    0.014488
(461, 544)    0.007020
(68, 487)     0.000000
(622, 932)    0.016355
(504, 936)    0.000000
